In [1]:
import os
if os.getcwd().endswith("experiments"):
    os.chdir("..")

os.getcwd()

'/home/lars/code/python/context-aware-flow-matching'

In [2]:
from torch.utils.data import random_split
from lightning import seed_everything

from lightning_trainable.utils import find_checkpoint

from src.models import Model
from src.datasets import ModelNet10Dataset

In [3]:
seed_everything(42)

Seed set to 42


42

In [4]:
dataset = ModelNet10Dataset("data/modelnet10", shapes=["chair"])

Found non-empty ModelNet10Dataset in data/modelnet10, skipping download...


Pre-Loading Meshes: 100%|█████████████████████████████████████████████████████████████████████| 989/989 [00:13<00:00, 73.40it/s]


In [5]:
train_data, val_data, test_data = random_split(dataset, [0.8, 0.1, 0.1])

In [6]:
hparams = dict(
    batch_size=256,
    subset_size=128,
    features=3,
    embeddings=512,
    accumulate_batches=1,
    checkpoint_segments=8,
    gamma=0.25,
    max_epochs=10_000,
    optimizer=dict(
        name="AdamW",
        kwargs=dict(
            weight_decay=0.01,
        ),
    ),
    lr_scheduler=dict(
        name="OneCycleLR",
        kwargs=dict(
            max_lr=1e-4,
            div_factor=1e1,
            final_div_factor=1e6,
        )
    ),
    gradient_clip=1.0,
    num_workers=14,
    accelerator="gpu",
    devices=-1,
)

In [ ]:
# initial training
model = Model(hparams, train_data, val_data, test_data)
model.fit(trainer_kwargs=dict(log_every_n_steps=1))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
The following callbacks returned in `LightningModule.configure_callbacks` will override existing callbacks passed to Trainer: ModelCheckpoint


#Layers in Encoder: 45
#Layers in Flow: 13


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                   | Params
---------------------------------------------------
0 | encoder | CheckpointedSequential | 23.9 M
1 | flow    | CheckpointedSequential | 14.4 M
---------------------------------------------------
38.3 M    Trainable params
0         Non-trainable params
38.3 M    Total params
153.197   Total estimated model params size (MB)


Sanity Checking: |                                                                                        | 0/…

Training: |                                                                                               | 0/…

...

Validation: |                                                                                             | 0/…

Validation: |                                                                                             | 0/…

Validation: |                                                                                             | 0/…

Validation: |                                                                                             | 0/…

Validation: |                                                                                             | 0/…

In [ ]:
# resume training
# cp = find_checkpoint(version=2)
# # cp = "lightning_logs/big-models/checkpoints/epoch=1849-step=14800.ckpt"
# model = Model(hparams, train_data, val_data, test_data)
# model.fit(trainer_kwargs=dict(log_every_n_steps=1, callbacks=model.configure_callbacks()), fit_kwargs=dict(ckpt_path=cp))